In [30]:
import os
import findspark
findspark.init(os.getenv('SPARK_HOME'))
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover, NGram

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/28 12:59:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
df = spark.createDataFrame([
                            (0, 'Hi I heard about Spark'),
                            (1, 'I wish java could use case classes'), 
                            (2, 'Logistic,Regression,models,are,neat')
                            ], ['id', 'sentence'])
df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,Regressi...|
+---+--------------------+



In [19]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [20]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [21]:
tokenized = tokenizer.transform(df)

In [22]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,Regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [23]:
rg_tokenized = regex_tokenizer.transform(df)
rg_tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,Regressi...|[logistic, regres...|
+---+--------------------+--------------------+



In [24]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,Regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [28]:
df2 = spark.createDataFrame([
                            (0, ['I', 'saw', 'the', 'green', 'horse']),
                            (1, ['Mary', 'had', 'a', 'little', 'lamb']), 
                            ], ['id', 'tokens'])
df2.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [29]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
remover.transform(df2).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [38]:
df3 = spark.createDataFrame([
                            (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
                            (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
                            (2, ['Logistic', 'Regression', 'models', 'are', 'neat'])
                            ], ['id', 'words'])
df3.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|[Hi, I, heard, ab...|
|  1|[I, wish, java, c...|
|  2|[Logistic, Regres...|
+---+--------------------+



In [40]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')
ngram.transform(df3).show(truncate=False)

+---+------------------------------------------+------------------------------------------------------------------+
|id |words                                     |grams                                                             |
+---+------------------------------------------+------------------------------------------------------------------+
|0  |[Hi, I, heard, about, Spark]              |[Hi I, I heard, heard about, about Spark]                         |
|1  |[I, wish, java, could, use, case, classes]|[I wish, wish java, java could, could use, use case, case classes]|
|2  |[Logistic, Regression, models, are, neat] |[Logistic Regression, Regression models, models are, are neat]    |
+---+------------------------------------------+------------------------------------------------------------------+

